In [1]:
using RigidBodyDynamics

In [2]:
using RigidBodyTreeInspector

In [3]:
mechanism = rand_chain_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1:5]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3
        Vertex: body4, Edge: joint4
          Vertex: body5, Edge: joint5
            Vertex: body6, Edge: joint6

In [4]:
vis = Visualizer(mechanism);

In [5]:
# We can draw the mechanism at a single state:
state = MechanismState(Float64, mechanism)
zero!(state)
setdirty!(state)
draw(vis, state)

In [6]:
# Or we can interactively explore the degrees of freedom of the mechanism.
# Note that the quaternion floating base joint, "joint1", has been given 
# six sliders. The first three correspond to the exponential map representation
# of its rotation, and the next three correspond to its translation. 
inspect(mechanism; show_inertias=true, randomize_colors=true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint1.1",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint1.2",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint1.3",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint1.4",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint1.5",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

...

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint3.1",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint4.1",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint5.1",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

Interact.Slider{Float64}(Signal{Float64}(-0.06411413578754688, nactions=0),"joint6.1",-0.06411413578754688,linspace(-3.141592653589793,3.141592653589793,50),true)

nothing

In [8]:
# we can also animate the mechanism, given a time sequence of states
# states will be linearly interpolated between the knot points
times = collect(linspace(0, 10, 3))
configurations = Vector{Vector{Float64}}(length(times))
for i = 1 : length(times)
    rand_configuration!(state)
    configurations[i] = configuration_vector(state)
end
animate(vis, mechanism, times, configurations; fps = 60., realtimerate = 1.)